<a href="https://www.kaggle.com/code/shoaibrkhan/trying-out-webscraping-imdb-top-10-000-movies?scriptVersionId=143642529" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep

from random import randint


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Creating the lists we want to write into
titles = []
years = []
time = []
genre = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []
headers = {'Accept-Language': 'en-US, en;q=0.5'}
pages = np.arange(1, 10505, 50)
pages

In [ ]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    url = 'https://www.imdb.com/search/title/?num_votes=10000,&sort=user_rating,desc&title_type=feature' + str(page) + '&ref_=adv_nxt'
    page = requests.get(url, headers=headers)

# soup = BeautifulSoup(response.content, 'html.parser')

#     page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)

        # Scraping the movie's length
        genre_m = container.find('span', class_='genre').text if container.p.find('span', class_='genre') else pd.NA
        genre.append(genre_m)

        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else pd.NA
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        
        grosses = nv[1].text if len(nv) > 1 else pd.NA
        us_gross.append(grosses)

In [ ]:
movies_df = pd.DataFrame({'movie':titles,
                       'year':years,
                       'genre':genre,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'vote':votes,
                       'gross_earning':us_gross})
movies_df

In [ ]:
movies_df.info()

In [ ]:
movies_df.to_csv('top_1000_voted_imdb_movies.csv', index=False)